In [1]:
import matplotlib
import warnings
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##資料讀取 - 查看

In [40]:
#查看會員訊息

df_cum = pd.read_excel('/content/drive/MyDrive/data analytics/department_store_sales_analysis/cumcm2018c1.xlsx')
df_cum

,会员卡号,出生日期,性别,登记时间
0,c68b20b4,2002-11-02 00:00:00,0.0,2013-05-11 00:00:00.000
1,1ca15332,NaN,0.0,2004-11-04 16:31:52.436
2,a37cc182,1967-02-17 00:00:00,0.0,2004-12-31 21:24:34.216
3,2ab88539,1982-06-01 00:00:00,0.0,2010-11-19 00:00:00.000
4,b4c77269,1964-02-05 00:00:00,0.0,2007-12-14 00:00:00.000
...,...,...,...,...
194755,a97474e0,NaN,NaN,NaT
194756,4e14ad22,NaN,0.0,2013-08-30 00:00:00.000
194757,f20299a8,NaN,NaN,2014-03-18 00:00:00.000
194758,a3a27f12,1973-01-10 00:00:00,0.0,2015-03-10 00:00:00.000


In [86]:
df_cum = df_cum.rename(columns={"会员卡号":"會員卡號",
                                "登记时间":"登記時間",
                                "性别":"性別"
                                })

df_cum

,會員卡號,出生日期,性別,登記時間,datetime,birth_year,birth_month,birth_day
0,c68b20b4,2002-11-02 00:00:00,0.0,2013-05-11 00:00:00.000,2002-11-02,2002.0,11.0,2.0
1,1ca15332,NaN,0.0,2004-11-04 16:31:52.436,NaT,NaN,NaN,NaN
2,a37cc182,1967-02-17 00:00:00,0.0,2004-12-31 21:24:34.216,1967-02-17,1967.0,2.0,17.0
3,2ab88539,1982-06-01 00:00:00,0.0,2010-11-19 00:00:00.000,1982-06-01,1982.0,6.0,1.0
4,b4c77269,1964-02-05 00:00:00,0.0,2007-12-14 00:00:00.000,1964-02-05,1964.0,2.0,5.0
...,...,...,...,...,...,...,...,...
194754,716746bc,NaN,NaN,2013-10-29 00:00:00.000,NaT,NaN,NaN,NaN
194756,4e14ad22,NaN,0.0,2013-08-30 00:00:00.000,NaT,NaN,NaN,NaN
194757,f20299a8,NaN,NaN,2014-03-18 00:00:00.000,NaT,NaN,NaN,NaN
194758,a3a27f12,1973-01-10 00:00:00,0.0,2015-03-10 00:00:00.000,1973-01-10,1973.0,1.0,10.0


In [87]:
#查看銷售表

df_sale = pd.read_csv('/content/drive/MyDrive/data analytics/department_store_sales_analysis//cumcm2018c2.csv', encoding = 'utf8')
df_sale

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,会员卡号,消费产生的时间,商品编码,销售数量,商品售价,消费金额,商品名称,此次消费的会员积分,收银机号,单据号,柜组编码,柜组名称
0,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,290.0,270.20,兰芝化妆品正价瓶,270.20,6,25bb,8077.0,兰芝柜
1,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,325.0,302.80,兰芝化妆品正价瓶,302.80,6,25bb,8077.0,兰芝柜
2,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,195.0,181.80,兰芝化妆品正价瓶,181.80,6,25bb,8077.0,兰芝柜
3,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,270.0,251.55,兰芝化妆品正价瓶,251.55,6,25bb,8077.0,兰芝柜
4,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,2,245.0,456.55,兰芝化妆品正价瓶,456.55,6,25bb,8077.0,兰芝柜
...,...,...,...,...,...,...,...,...,...,...,...,...
1893527,607c7cde,2018-01-03 21:38:15.343,c483e08b,1,200.0,200.00,植村秀眉笔4g,200.00,91,3d84,8156.0,Shu Uemura
1893528,220c0564,2018-01-03 21:41:53.140,cd93b1ca,1,1.0,1.00,爱慕内衣正价件,1.00,264,7cd8,4332.0,Aimer
1893529,220c0564,2018-01-03 21:41:53.140,cd93b1ca,1,490.0,490.00,爱慕内衣正价件,490.00,264,7cd8,4332.0,Aimer
1893530,220c0564,2018-01-03 21:41:53.140,cd93b1ca,1,280.0,280.00,爱慕内衣正价件,280.00,264,7cd8,4332.0,Aimer


In [88]:
df_sale = df_sale.rename(columns={"会员卡号":"會員卡號",
                                  "消费产生的时间":"消費時間",
                                  "商品编码":"商品編碼",
                                  "销售数量":"銷售數量",
                                  "商品售价":"商品售價",
                                  "消费金额":"消費金額",
                                  "商品名称":"商品名稱",
                                  "此次消费的会员积分":"此次消費的會員積分",
                                  "收银机号":"收銀機號",
                                  "单据号":"單據號碼",
                                  "柜组编码":"專櫃編碼",
                                  "柜组名称":"專櫃名稱"                                                   
                                  })
df_sale

,會員卡號,消費時間,商品編碼,銷售數量,商品售價,消費金額,商品名稱,此次消費的會員積分,收銀機號,單據號碼,專櫃編碼,專櫃名稱
0,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,290.0,270.20,兰芝化妆品正价瓶,270.20,6,25bb,8077.0,兰芝柜
1,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,325.0,302.80,兰芝化妆品正价瓶,302.80,6,25bb,8077.0,兰芝柜
2,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,195.0,181.80,兰芝化妆品正价瓶,181.80,6,25bb,8077.0,兰芝柜
3,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,270.0,251.55,兰芝化妆品正价瓶,251.55,6,25bb,8077.0,兰芝柜
4,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,2,245.0,456.55,兰芝化妆品正价瓶,456.55,6,25bb,8077.0,兰芝柜
...,...,...,...,...,...,...,...,...,...,...,...,...
1893527,607c7cde,2018-01-03 21:38:15.343,c483e08b,1,200.0,200.00,植村秀眉笔4g,200.00,91,3d84,8156.0,Shu Uemura
1893528,220c0564,2018-01-03 21:41:53.140,cd93b1ca,1,1.0,1.00,爱慕内衣正价件,1.00,264,7cd8,4332.0,Aimer
1893529,220c0564,2018-01-03 21:41:53.140,cd93b1ca,1,490.0,490.00,爱慕内衣正价件,490.00,264,7cd8,4332.0,Aimer
1893530,220c0564,2018-01-03 21:41:53.140,cd93b1ca,1,280.0,280.00,爱慕内衣正价件,280.00,264,7cd8,4332.0,Aimer


##數據預處理
1.處理會員資料
  * 檢查是否重複
  * 將出生日期新增成datetime資料
  * 將登記時間新增成datetime資料  

2.處理銷售資料  
  * 觀察資料
  * 會員資料合併至此，將會員做分類(非會員)
  * LRFM
  * 顧客特徵文字雲
  


In [89]:
print(df_cum)
print(df_cum.shape)

#19萬筆 四個特徵 可觀察到皆有缺失值

            會員卡號                 出生日期   性別                    登記時間   datetime  \
0       c68b20b4  2002-11-02 00:00:00  0.0 2013-05-11 00:00:00.000 2002-11-02   
1       1ca15332                  NaN  0.0 2004-11-04 16:31:52.436        NaT   
2       a37cc182  1967-02-17 00:00:00  0.0 2004-12-31 21:24:34.216 1967-02-17   
3       2ab88539  1982-06-01 00:00:00  0.0 2010-11-19 00:00:00.000 1982-06-01   
4       b4c77269  1964-02-05 00:00:00  0.0 2007-12-14 00:00:00.000 1964-02-05   
...          ...                  ...  ...                     ...        ...   
194754  716746bc                  NaN  NaN 2013-10-29 00:00:00.000        NaT   
194756  4e14ad22                  NaN  0.0 2013-08-30 00:00:00.000        NaT   
194757  f20299a8                  NaN  NaN 2014-03-18 00:00:00.000        NaT   
194758  a3a27f12  1973-01-10 00:00:00  0.0 2015-03-10 00:00:00.000 1973-01-10   
194759  a03cc001  1964-08-11 00:00:00  0.0 2010-08-09 00:00:00.000 1964-08-11   

        birth_year  birth_m

In [90]:
df_cum.loc[:,'出生日期']

0         2002-11-02 00:00:00
1                         NaN
2         1967-02-17 00:00:00
3         1982-06-01 00:00:00
4         1964-02-05 00:00:00
                 ...         
194754                    NaN
194756                    NaN
194757                    NaN
194758    1973-01-10 00:00:00
194759    1964-08-11 00:00:00
Name: 出生日期, Length: 182070, dtype: object

In [91]:
print('會員資料表一共有{}行，{}列'.format(df_cum.shape[0], df_cum.shape[1]))
print('數據缺失的平均值為：\n{}'.format(df_cum.isnull().mean()))
print('會員卡號（不重複）有{}筆'.format(len(df_cum['會員卡號'].unique())))

print(f'重複的會員資料有{194760-194754}筆')

會員資料表一共有182070行，8列
數據缺失的平均值為：
會員卡號           0.000000
出生日期           0.151568
性別             0.016587
登記時間           0.000000
datetime       0.151656
birth_year     0.151656
birth_month    0.151656
birth_day      0.151656
dtype: float64
會員卡號（不重複）有182070筆
重複的會員資料有6筆


In [92]:
# 將日期欄位轉成datatime物件
df_cum['datetime'] = pd.to_datetime(df_cum['出生日期'],errors = 'coerce') 
#如果日期格式有錯誤轉換成datatime會報錯，可以使用errors = 'coerce' 把錯誤的格式轉換成Nat

df_cum['birth_year'] = df_cum['datetime'].dt.year
df_cum['birth_month'] = df_cum['datetime'].dt.month
df_cum['birth_day'] = df_cum['datetime'].dt.day

In [93]:
print(f"出生年份缺失的比例為：{round(df_cum['birth_year'].isnull().sum() / df_cum['birth_year'].count() * 100,2)}%")
print(f"出生月份缺失的比例為：{round(df_cum['birth_month'].isnull().sum() / df_cum['birth_month'].count() * 100,2)}%")
print(f"出生日期缺失的比例為：{round(df_cum['birth_day'].isnull().sum() / df_cum['birth_day'].count() * 100,2)}%")

#缺失比例過高，LRFM可能會失去精準度

出生年份缺失的比例為：17.88%
出生月份缺失的比例為：17.88%
出生日期缺失的比例為：17.88%


In [94]:
#會員卡號去除重覆值

df_cum.drop_duplicates(subset='會員卡號',inplace=True)
print('會員卡號去除重複後，共有{}筆unique'.format(len(df_cum['會員卡號'].unique())))

會員卡號去除重複後，共有182070筆unique


In [95]:
#登記時間缺失值刪除 (無法做RFM分析)

df_cum.dropna(subset = ['登記時間'], inplace=True)
print(f'去除重複會員以及入會時間後，資料shape{df_cum.shape}')

去除重複會員以及入會時間後，資料shape(182070, 8)


In [107]:
print(df_cum['性別'].value_counts()) #0女性、1男性
print(f"性別缺失值數量為：{df_cum['性別'].isnull().sum()}")

0.0    144290
1.0     34760
Name: 性別, dtype: int64
性別缺失值數量為：3020


In [108]:
#mode()方法可以找出欄位中最常見的值 回傳一個series
df_cum['性別'].fillna(df_cum['性別'].mode().values[0], inplace = True)
df_cum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182070 entries, 0 to 194759
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   會員卡號         182070 non-null  object        
 1   出生日期         154474 non-null  object        
 2   性別           182070 non-null  float64       
 3   登記時間         182070 non-null  datetime64[ns]
 4   datetime     154458 non-null  datetime64[ns]
 5   birth_year   154458 non-null  float64       
 6   birth_month  154458 non-null  float64       
 7   birth_day    154458 non-null  float64       
dtypes: datetime64[ns](2), float64(4), object(2)
memory usage: 12.5+ MB


In [110]:
#檢查登記時間(入會時間) 是否存在異常值，若有異常值無法進行基礎運算操作

df_test = df_cum['登記時間'] + pd.Timedelta(days = 1)
df_test

#登記時間序列正常

0        2013-05-12 00:00:00.000
1        2004-11-05 16:31:52.436
2        2005-01-01 21:24:34.216
3        2010-11-20 00:00:00.000
4        2007-12-15 00:00:00.000
                   ...          
194754   2013-10-30 00:00:00.000
194756   2013-08-31 00:00:00.000
194757   2014-03-19 00:00:00.000
194758   2015-03-11 00:00:00.000
194759   2010-08-10 00:00:00.000
Name: 登記時間, Length: 182070, dtype: datetime64[ns]

In [111]:
# 查看處理後數據的缺失狀況
df_cum.isnull().mean()

會員卡號           0.000000
出生日期           0.151568
性別             0.000000
登記時間           0.000000
datetime       0.151656
birth_year     0.151656
birth_month    0.151656
birth_day      0.151656
dtype: float64

In [119]:
#出生日期包含大量缺失值，且存在異常值 新建一個dataframe來保存出生日期和性別，方便對會員的性別和年齡訊息進行統計

new_df_cum = pd.DataFrame(df_cum.loc[df_cum['出生日期'].notnull(),['出生日期', '性別']])
new_df_cum['年齡'] = new_df_cum['出生日期'].astype(str).apply(lambda x : x[:3] + '0') #這邊把每10年當作一個單位，方便後續統計
new_df_cum.drop('出生日期', axis = 1, inplace = True)
new_df_cum['年齡'].value_counts()

1980    47142
1970    43407
1960    26678
1750    16748
1990    11811
1950     7078
1940      893
2010      370
2000      139
1930      110
1920       32
1900       21
1910        8
1880        4
1890        3
1860        3
5220        2
1870        2
1760        2
1850        2
9790        1
1790        1
2640        1
1780        1
8500        1
8900        1
7710        1
5190        1
9740        1
1770        1
5010        1
2190        1
9820        1
9880        1
1820        1
8400        1
8700        1
9690        1
6200        1
Name: 年齡, dtype: int64

In [124]:
#出生年份出現較多異常值 檢查登記時間觀察最後入會正常時間為2018年
df_test = df_cum['登記時間'] + pd.Timedelta(days = 1)
df_test.sort_values()

#若以年齡100歲來說，2018-100 = 1918 故1910以下可能資料有誤或其他因素，假設會員年齡範圍在1910-2020之間，將超過範圍的值當作異常值刪除

new_df_cum['年齡']  = new_df_cum['年齡'].astype(int)
condition = "年齡 >= 1920 and 年齡<= 2020"
new_df_cum = new_df_cum.querry(condition)

149219   1900-01-02 00:00:00.000
748      2002-02-26 00:00:00.000
136740   2002-10-24 21:10:35.260
6520     2002-10-25 11:06:39.980
72981    2002-10-25 11:27:08.440
                   ...          
129216   2018-01-05 10:19:36.210
68959    2018-01-05 10:42:06.560
68960    2018-01-05 11:03:00.336
151323   2080-01-18 00:00:00.000
9280     2080-07-04 00:00:00.000
Name: 登記時間, Length: 182070, dtype: datetime64[ns]

RFM 模型是一種客戶價值分析模型，將客戶價值分成三個維度：近期銷售（R）、頻率（F）和金額（M）。

近期銷售（R）表示客戶上次購買的時間距今的天數。客戶越新鮮，其價值就越高。

頻率（F）表示客戶在一定時間內購買次數。客戶購買次數越高，其價值就越高。

金額（M）表示客戶在一定時間內為公司帶來的總收益。

通過對這三個維度的評估，可以更好地了解客戶的價值，並為公司提供有效的營銷策略。

通常我們可以將顧客分為精英、重要、一般、潛在、流失五種類型，根據不同類型採用不同的營銷策略。